<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week15/simple_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Chatbot

In this final part, we will take the models that we have trained and use them to recognize an intent and the entities and building simple responses to that. 

Before starting, click on the Colab's Runtime > Manage Sessions menu. Click the "TERMINATE OTHER SESSIONS" button.  

Then, run the following cells to download the models (after training them to our intent and token classification tasks) and install the necessary libraries. 

The reason we are doing this is because downloading the models that you have trained from Colab is VERY slow. So we've already saved a copy of our own trained models and uploaded it to a public server on Amazon for download.


In [ ]:
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/intent_model.zip
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/token_model.zip
!unzip intent_model.zip
!unzip token_model.zip


Next, run the following to install a specific version of the HuggingFace Transformers library.

Our model was trained against this version of the library, so it is advisable to use the same version for prediction / inference.

In [ ]:
!pip install transformers

## Section 1 - Inferring Intent

In this section, we declare the codes to infer intent based on a single line of input text.

In [ ]:
# Import the necessary libraries
#
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification
)

import numpy as np
import tensorflow as tf


# Create the DistilBERT tokenizer
#
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


# Create a list of unique labels that we will recognize.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# Define a function to perform inference on a single input text.
# 
def infer_intent(text, model, tokenizer):
    # Passes the text into the tokenizer
    #
    input = tokenizer(text, truncation=True, padding=True, return_tensors="tf")

    # Sends the result from the tokenizer into our classification model
    #
    output = model(input)
    pred_label = np.argmax(tf.nn.softmax(output.logits, axis=-1))

    # Return the result to the caller
    #
    return sentence_labels[pred_label]


# Load the saved model file
#
intent_model = TFAutoModelForSequenceClassification.from_pretrained('intent_model')



Run the following cell to test the codes that infers the intent. 

In [ ]:
infer_intent("How much is the ticket to fly to New York", intent_model, tokenizer)

## Section 2 - Inferring Entity

In this section, we declare the codes to infer entities for each individual word in a line of text. The entities are then constructed and returned to the caller.


In [ ]:
from transformers import (
    AutoTokenizer,
    TFAutoModelForTokenClassification
)
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# Define a list of unique labels that we will recognized
#
token_labels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Define the function to infer the individual tokens
#
def infer_tokens(text, model, tokenizer):
    # here we assume the text has not been splitted into individual words
    text = text.split()
    
    encodings = tokenizer(
        [text],
        padding=True,
        truncation=True,
        is_split_into_words=True,
        return_tensors='tf')
    
    logits = model(encodings)[0] # assume only a single prediction
    preds = np.argmax(logits, axis=-1)[0]

    # as the prediction is on individual tokens, including subtokens, 
    # we need to group subtokens belonging to the same word together
    # again, we use the word_ids to help us here
    previous_word_idx = None
    word_ids = encodings[0].word_ids
    labels = []
    for i, word_idx in enumerate(word_ids):
        # we check if the word_id different from previous one, then it is a new word
        # we also need to check if the word_id is not None so that we won't include it
        if word_idx != previous_word_idx and word_idx != None:
            labels.append(label_names[preds[i]])
        # update the previous_word_idx to current word_id
        previous_word_idx = word_idx

    return text, labels


# Define the function to combine individual tokens into a dictionary
#
def infer_combined_tokens(text, token_model, tokenizer):
    result = {
        "PER" : [],
        "LOC" : [],
        "ORG" : [],
        "MISC" : []
    }

    result_tokens, result_texts = infer_tokens(text, token_model, tokenizer)

    current_token_label = ""
    current_result_index = -1;    

    for i in range(len(result_tokens)):
        #print(result_tokens[i])
        if token_labels[result_tokens[i]].startswith("B-"):
            current_token_label = token_labels[result_tokens[i]].replace("B-", "")
            result[current_token_label].append(result_texts[i])
            current_result_index = len(result[current_token_label]) - 1
        elif token_labels[result_tokens[i]].startswith("I-"):
            result[current_token_label][current_result_index] += " " + result_texts[i]
    
    return result


In [ ]:
token_model = TFAutoModelForTokenClassification.from_pretrained('token_model')

In [ ]:
text, tokens = infer_tokens("How much is the ticket to fly to New York", token_model, tokenizer)

In [ ]:
print(tokens)
print(text)

Run the following cell to test the codes that extracts and combines all the entities for us.

In [ ]:
infer_combined_tokens("Peter Leong and John Lim of Aims are going to fly to New York", 
                      token_model, 
                      tokenizer)

## Section 3 - Implementing Logic for Our Chatbot

In this section, let's implement some very basic logic for our chatbot. We will make use of the two functions that we wrote above.

You can implement some simple logic that looks like the following:

```
        if (intent == "atis_flight" or intent == "atis_airline") and len(tokens["LOC"]):
            print ("Can I confirmed if you just asked about flying to " + tokens["LOC"][0])
        elif intent == "atis_yes":
            print ("Great, then let's me book the ticket for you")
        elif intent == "atis_no":
            print ("Oh I am sorry what did I get wrong?")
        elif intent == "atis_greeting":
            print ("Hi, how are you?")            
        else:
            print ("I don't quite know how to respond to " + intent + " yet.")
```

In [ ]:
def chatbot():
    print ("Chatbot Started. Press 'Q'+Enter to quit.")

    while (True):
        input_text = input()
        if input_text == "Q" or input_text == "":
            break

        intent = infer_intent(intent_model, input_text)
        tokens = infer_combined_tokens(token_model, input_text)

        # TODO: 
        # Write you own logic to conduct a conversation with the user
        # about buying tickets and flying somewhere.
        #...#
        if (intent == "atis_flight" or intent == "atis_airline") and len(tokens["LOC"]):
            print ("Can I confirmed if you just asked about flying to " + tokens["LOC"][0])
        elif intent == "atis_yes":
            print ("Great, then let me book the ticket for you")
        elif intent == "atis_no":
            print ("Oh I am sorry what did I get wrong?")
        elif intent == "atis_greeting":
            print ("Hi, how are you?")            
        else:
            print ("I don't quite know how to respond to " + intent + " yet.")

    print ("Good bye!")

chatbot()
